In [1]:
import logging
import requests
import json
import time
import pandas as pd
from datetime import datetime
from web3 import Web3
from decouple import config


In [ ]:
import json
import time
import pandas as pd
from datetime import datetime
from web3 import Web3

In [ ]:
Etherscan_APIKEY = config('ETHERSCAN_APIKEY')
RPC_Endpoint = config('RPC_MAINNET')

In [ ]:
w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())

In [ ]:
# Convex Pools dictionary
#https://github.com/convex-eth/utilities/blob/79852a01f9d2cda169fcbc246569057ee9ba0161/sheetScripts/convex.gs#L609
GithubConvexPoolData = "https://raw.githubusercontent.com/convex-eth/utilities/main/sheetScripts/convex.gs" 
data = requests.get(GithubConvexPoolData).text
pos_start = data.index('const pools =')
pools_txt = data[pos_start:]
pos_end = pools_txt.index(';')
pools_txt = pools_txt[:pos_end]
pools_txt = pools_txt.replace('const pools =','')

pools = json.loads(pools_txt)

display(pd.DataFrame.from_dict(pools))

In [ ]:
abi_cache = {}
CurveBaseApysCache = {}

def pull_abi_etherscan(contract_address, apikey = Etherscan_APIKEY):
    url = 'https://api.etherscan.io/api?module=contract&action=getabi'
    params = {'address':contract_address,'apikey' : apikey}
            
    if contract_address in abi_cache:
        logging.info(f'Pulling ABI for contract_address: {contract_address} -  return from cache') 
        abi =  abi_cache[contract_address]
    else:
        logging.info(f'Pulling ABI for contract_address: {contract_address} - get ABI from etherscan')
        if not apikey:
            logging.info(f'No Etherscan API key, waiting 5 sec to avoid rate limit...') 
            time.sleep(5) # rate-limit when apikey is empty

        response = requests.get(url, params=params)
        response_json = json.loads(response.text)

        logging.info(f'Pulling done') 

        if response_json['status']  == '1':
            abi = json.loads(response_json['result'])
            abi_cache[contract_address] = abi
        else:
            raise Exception(response_json['result'])
    
    
    return abi

def getCurveBaseApys(name):
    if name in CurveBaseApysCache:
        baseApy = float(CurveBaseApysCache[name]['baseApy'])
    else:    
        response = requests.get('https://www.convexfinance.com/api/curve-apys')
        response = json.loads(response.text)
        CurveBaseApysCache.update(response['apys'])
    
    baseApy = float(CurveBaseApysCache[name]['baseApy'])    
    baseApy/=100
    return baseApy

In [ ]:
crvAddress = "0xD533a949740bb3306d119CC777fa900bA034cd52"
cvxAddress = "0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B"
cvxCrvAddress = "0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7"

cliffSize = 100000 # * 1e18; //new cliff every 100,000 tokens
cliffCount = 1000 # 1,000 cliffs
maxSupply = 100000000 # * 1e18; //100 mil max supply

def rewardRate(contract, contract_abi=None):
    if not contract_abi:
        contract_abi = pull_abi_etherscan(contract)
    
    hexValue = w3_eth.eth.contract(address=contract, abi=contract_abi).functions.rewardRate().call()
    rewardRate = hexValue/10**18
    
    return rewardRate

def supplyOf(contract, contract_abi=None):
    if not contract_abi:
        contract_abi = pull_abi_etherscan(contract)
        
    hexValue = w3_eth.eth.contract(address=contract, abi=contract_abi).functions.totalSupply().call()
    supplyOf = hexValue/10**18
    
    return supplyOf

def balanceOf(address, contract, decimals, contract_abi=None):
    if not contract_abi:
        contract_abi = pull_abi_etherscan(contract)
    
    balanceOf = w3_eth.eth.contract(address=contract, abi=contract_abi).functions.balanceOf(address).call()
    balanceOf = balanceOf/10**decimals

    return balanceOf

def getPrice(contract_address, vsCoin):
    url = 'https://api.coingecko.com/api/v3/simple/token_price/ethereum'
    params = {'contract_addresses':contract_address,
              'vs_currencies': vsCoin,
             }
    response = requests.get(url, params=params)
    response = json.loads(response.text)
    logging.info(f'getPrice({contract_address},{vsCoin}) response {response}') 
    response = response[contract_address.lower()][vsCoin.lower()]
    
    return response

def curveLpValue(amount, swapAddress, swapAddress_abi = None):
    if not swapAddress_abi:
        swapAddress_abi = pull_abi_etherscan(swapAddress)
        
    hexValue = w3_eth.eth.contract(address=swapAddress, abi=swapAddress_abi).functions.get_virtual_price().call() 
    pricePerShare = hexValue/10**18
    
    return amount * pricePerShare;

def curveV2LpValue(pool, currencyType):
    #get amount of tokens
    supply = supplyOf(pool['lptoken'])
    logging.info(f'supply: {supply}')
    
    total = 0
    for i in range(len(pool['coins'])):
        bal = balanceOf(pool['swap'], pool['coins'][i], pool['coinDecimals'][i])
        logging.info(f'bal: {i} = {bal}')
        
        price = getPrice(pool['coins'][i], currencyType)
        logging.info(f'price: {i} = {price}') 

        total += (bal*price)
  
    value = total/supply
                      
    return value

def getCVXMintAmount(crvEarned):
    #first get total supply
    cvxSupply = supplyOf(cvxAddress)
    #get current cliff
    currentCliff = cvxSupply / cliffSize
    #if current cliff is under the max
    if currentCliff < cliffCount:
        #get remaining cliffs
        remaining = cliffCount - currentCliff

        #multiply ratio of remaining cliffs to total cliffs against amount CRV received
        cvxEarned = crvEarned * remaining / cliffCount

        #double check we have not gone over the max supply
        amountTillMax = maxSupply - cvxSupply
        if cvxEarned > amountTillMax:
            cvxEarned = amountTillMax;
            
        return cvxEarned

    return 0
def convexAPR(poolName):
    return convexAPRWithPrice(poolName, -1, -1)

def convexAPRWithPrice(poolName, crvPrice, cvxPrice):
    
    baseAPR = getCurveBaseApys(poolName)
    logging.info(f'Base Curve vAPR: {baseAPR}')
    
    pool = [x for x in pools if x['name']==poolName][0]
    curveSwap = pool['swap']
    stakeContract = pool['crvRewards']

    #get reward rate
    rate = rewardRate(stakeContract)
    
    #get virtual price
    virtualPrice = 1;
    if not 'isV2' in pool or not pool['isV2']:
        virtualPrice = curveLpValue(1, curveSwap)
    else:
        virtualPrice = curveV2LpValue(pool, pool['currency'])

    #get supply
    supply = supplyOf(stakeContract);

    #virtual supply
    supply = supply * virtualPrice;

    #crv per underlying per second
    crvPerUnderlying = rate / supply;

    #crv per year
    crvPerYear = crvPerUnderlying * 86400 * 365;
    cvxPerYear = getCVXMintAmount(crvPerYear);

    if cvxPrice <= 0:
        cvxPrice = getPrice(cvxAddress, pool['currency'])

    if crvPrice <= 0:
        crvPrice = getPrice(crvAddress, pool['currency'])
    
    crvAPR = (crvPerYear * crvPrice)
    logging.info(f'apr of crv: {crvAPR}')
    
    cvxAPR = (cvxPerYear * cvxPrice)
    logging.info(f'apr of cvx: {cvxAPR}')
    
    extraAPR = 0
    if 'extras' in pool and len(pool['extras']) > 0:
        for ex in pool['extras']:
            exrate = rewardRate(ex['contract'])
            perUnderlying = exrate / supply
            perYear = perUnderlying * 86400 * 365
            price = getPrice(ex['token'], pool['currency'])
            
            logging.info(f'extra per year: {perYear} price: {price} apr: {perYear*price}') 
            extraAPR += (perYear * price)
    
    apr = baseAPR + crvAPR + cvxAPR + extraAPR
    logging.info(f'apr of base/crv/cvx/extra: {apr}') 
    
    return {'APR':apr,'baseAPR':baseAPR,  'crvAPR':crvAPR, 'cvxAPR':cvxAPR, 'extraAPR': extraAPR}


#staked CVX APR
def convexStakedCVXAPR():
    stakeContract = "0xCF50b810E57Ac33B91dCF525C6ddd9881B139332"
    rate = rewardRate(stakeContract)
    supply = supplyOf(stakeContract)
    cvxPrice = getPrice(cvxAddress, "USD")
    supply *= cvxPrice
    rate /= supply

    crvPerYear = rate * 86400 * 365
    crvPrice = getPrice(crvAddress, "USD")
    apr = crvPerYear * crvPrice

    return apr

#staked cvxCRV apr
def convexStakedCvxCRVAPR():
    stakeContract = "0x3Fe65692bfCD0e6CF84cB1E7d24108E434A7587e"
    theepoolstakeContract = "0x7091dbb7fcbA54569eF1387Ac89Eb2a5C9F6d2EA"
    curveSwap = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"

    rate = rewardRate(stakeContract)
    threerate = rewardRate(theepoolstakeContract)
    supply = supplyOf(stakeContract)

    virtualPrice = curveLpValue(1, curveSwap)
    crvPrice = getPrice(crvAddress, "USD")
    cvxPrice = getPrice(cvxAddress, "USD")

    supply *= crvPrice
    rate /= supply
    threerate /= supply

    crvPerYear = rate * 86400 * 365
    cvxPerYear = getCVXMintAmount(crvPerYear)
    threepoolPerYear = threerate * 86400 * 365

    apr = (crvPerYear * crvPrice) + (cvxPerYear * cvxPrice) + (threepoolPerYear * virtualPrice)

    return apr



In [ ]:
#enable logging for debug
#logging.basicConfig(
#        format='%(asctime)s %(levelname)-8s %(message)s',
#        level='INFO',
#        datefmt='%Y-%m-%d %H:%M:%S',
#        )

# How to use:
- Curve LP APR 
`=convexAPR("poolname")`

- Staked CVX APR 
`=convexStakedCVXAPR()`

- Staked cvxCRV APR: 
`= convexStakedCvxCRVAPR()`

In [ ]:
print(datetime.utcnow(), 'start...')

print('frax pool:')
fraxPoolAPR = convexAPR('frax')
print(fraxPoolAPR)

print('\nsteth pool:')
stethPoolAPR = convexAPR('steth')
print(stethPoolAPR)

print('\nlusd pool:')
lusdPoolAPR = convexAPR('lusd')
print(lusdPoolAPR)

print('\ntricrypto2 pool:')
tricrypto2APR = convexAPR('tricrypto2')
print(tricrypto2APR)

In [ ]:
StakedCVXAPR = convexStakedCVXAPR()
print('Staked CVX apr', StakedCVXAPR)

StakedCvxCRVAPR = convexStakedCvxCRVAPR()
print('Staked CvxCRV apr', StakedCvxCRVAPR)
